In [1]:
import pandas
import json
import openpyxl
from openpyxl import load_workbook

In [2]:
# Using data from mtco2_2019.xlsx
workbook = load_workbook("mtco2_2019.xlsx")
co2_data_sheet = workbook.active

rows = []
for row in co2_data_sheet.iter_rows(min_row=2, max_row=3, min_col=2, max_col=222):
    rows.append(row)
    

row_1 = list(rows[0])
row_2 = list(rows[1])

country_list = [item.value for item in row_1]
territorial_co2 = [item.value for item in row_2]

emissions = [[country, emission] for country, emission in zip(country_list, territorial_co2)]

In [3]:
emissions_2019 = dict(emissions)

emissions_2019['Russia'] = emissions_2019.pop('Russian Federation')
emissions_2019['United States'] = emissions_2019.pop('United States of America')

In [4]:
# Using data from world.json
with open("world.json", "r", encoding='utf-8-sig') as file_handle:
    info = json.load(file_handle)

countries_mapping = dict(info)

In [5]:
# Adding co2 data to world.json
def add_co2_values(i):
    country_name = countries_mapping['features'][i]['properties']['NAME']
    if country_name not in emissions_2019.keys() or emissions_2019[country_name] is None:
        countries_mapping['features'][i]['properties']['EMISSIONS'] = float(0)
    else:
        countries_mapping['features'][i]['properties']['EMISSIONS'] = float(emissions_2019[country_name])

In [6]:
len(countries_mapping['features'])

246

In [7]:
# Applying function add_co2_values(i) on rows 0-245
for num in range(246):
    add_co2_values(num)

In [8]:
# Converting final dictionary into a JSON file
with open("total_world_co2_2019.json", "w+") as file_handle:
    json.dump(countries_mapping, file_handle)